# E2B Python SDK
The E2B Python SDK provides an interface for managing cloud environments for AI agents.

This SDK gives your agent a full cloud development environment that's sandboxed. That means:

- Access to Linux OS
- Using filesystem (create, list, and delete files and dirs)
- Run processes
- Sandboxed - you can run any code
- Access to the internet

These cloud environments are meant to be used for agents. Like a sandboxed playgrounds, where the agent can do whatever it wants.


## Installation

In [1]:
pip install e2b==0.1.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: e2b
    Found existing installation: e2b 0.1.3
    Uninstalling e2b-0.1.3:
      Successfully uninstalled e2b-0.1.3

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Usage

In [2]:
from e2b import Sandbox

id = "Nodejs"

### Initialization

In [5]:
# You can use some of the predefined environments by using specific id:
# 'Nodejs', 'Bash', 'Python3', 'Java', 'Go', 'Rust', 'PHP', 'Perl', 'DotNET'
sandbox = Sandbox.create(id)

# Close the session after you are done
sandbox.close()

### Filesystem

In [13]:
# Create, read and delete file

sandbox = Sandbox.create(id)

sandbox.filesystem.write("/tmp/test.txt", "Hello World!")

content = sandbox.filesystem.read("/tmp/test.txt")
print(content)

sandbox.filesystem.remove("/tmp/test.txt")

sandbox.close()

Hello World!


In [14]:
# List files in a directory, create a new directory and remove a directory

sandbox = Sandbox.create(id)

ls = sandbox.filesystem.list("/")
print([x.name for x in ls if x.name.startswith("test")])

sandbox.filesystem.make_dir("/test/new")

ls = sandbox.filesystem.list("/")
print([x.name for x in ls if x.name.startswith("test")])

sandbox.filesystem.remove("/test")

ls = sandbox.filesystem.list("/")
print([x.name for x in ls if x.name.startswith("test")])

sandbox.close()

[]
['test']
[]


In [15]:
# Watch directory for changes (WIP)

sandbox = Sandbox(id)
sandbox.open()

watcher = sandbox.filesystem.watch_dir("/")
watcher.add_event_listener(lambda event: print("Event", event))
watcher.start()

sandbox.filesystem.write("/test.txt", "Hello World!")

watcher.stop()

sandbox.close()

Event {'path': '//test.txt', 'name': 'test.txt', 'operation': 'Create', 'timestamp': 1691532768257138654, 'isDir': False}
Event {'path': '//test.txt', 'name': 'test.txt', 'operation': 'Write', 'timestamp': 1691532768259136251, 'isDir': False}


### Process

In [3]:
# Execute a command and get the output

sandbox = Sandbox.create(id)

proc = sandbox.process.start(
    "pwd",
    # If you add a callback for stdout you will get the stderr data as it comes in. 
    # You can still access the stdout after `output = proc.wait()` as `output.stdout` or anytime as `proc.output.stdout`.
    on_stdout=lambda data: print("Stdout", data),
    # If you add a callback for stderr you will get the stderr data as it comes in.
    # You can still access the stderr after `output = proc.wait()` as `output.stderr` or anytime as `proc.output.stderr`.
    on_stderr=lambda data: print("Stderr", data),
    on_exit=lambda: print("Exit"),
    cwd="/code",
)
print("Process session ID", proc.process_id)

# You can access `proc.output...` even before the process finishes
print("Current output", proc.output.messages)

# You can wait for the process to end
output = proc.wait()

# List of all output messages
print("Output", output.messages)
# Concatenated stdout
print("Stdout", output.stdout)
# Concatenated stderr
print("Stderr", output.stderr)
sandbox.close()

Process session ID wI6N300SFSvq
Current output []
Stdout line='/code' error=False timestamp=1691532768326208600
Exit
Output [ProcessMessage(line='/code', error=False, timestamp=1691532768326208600)]
Stdout /code
Stderr 


In [9]:
# Start command, send stdin to it and then kill it

sandbox = Sandbox.create(id)

proc = sandbox.process.start(
    "while IFS= read -r line; do echo \"$line\"; sleep 1; done",
    # If you add a callback for stdout you will get the stderr data as it comes in. 
    # You can still access the stdout after `output = proc.wait()` as `output.stdout` or anytime as `proc.output.stdout`.
    on_stdout=lambda data: print("Stdout", data),
    # If you add a callback for stderr you will get the stderr data as it comes in.
    # You can still access the stderr after `output = proc.wait()` as `output.stderr` or anytime as `proc.output.stderr`.
    on_stderr=lambda data: print("Stderr", data),
    on_exit=lambda: print("Exit"),
    cwd="/code",
)
proc.send_stdin("marco\n")
proc.kill()

print("Output", proc.output.messages)
print("Stdout", proc.output.stdout)
print("Stdout", proc.output.stderr)

sandbox.close()

Stdout line='marco' error=False timestamp=1691532768254368795
Exit
Output [ProcessMessage(line='marco', error=False, timestamp=1691532768254368795)]
Stdout marco
Stdout 


### Terminal

In [19]:
# Start and interact with a terminal session

sandbox = Sandbox.create(id)

term = sandbox.terminal.start(
    on_data=lambda data: print(data),
    on_exit=lambda: print("Exit"),
    cols=80,
    rows=24,
    cwd="/code",
)
print("Terminal session ID", term.terminal_id)
term.resize(80, 30)

term.send_data("ls -a\n")

term.kill()

sandbox.close()

Terminal session ID FSSpmurO9BNP

/code $ 
ls -a

.  ..  package.json

/code $ 
Exit


In [16]:
# Execute one command inside a terminal session

sandbox = Sandbox.create(id)

term = sandbox.terminal.start(
    on_data=lambda data: print("Data:", data),
    on_exit=lambda: print("Exit"),
    cols=80,
    rows=24,
    cwd="/code",
    # If you specify a command, the terminal will be closed after the command finishes.
    cmd="echo Hello World",
)

term.wait()

sandbox.close()

Data: Hello World

Exit


### Ports and hostnames - connecting to environment

In [17]:
# Get notified when a port opens and how to get a public hostname for an open port in the cloud environment
import time

sandbox = Sandbox.create(
    id,
    on_scan_ports=lambda ports: print("Open ports:", [port.port for port in ports]),
)

port = 8000
proc = sandbox.process.start(f"python3 -m http.server {port}")

hostname = sandbox.get_hostname(port)
print(f"Hostname: https://{hostname}")

time.sleep(10)

proc.kill()

sandbox.close()

Hostname: https://8000-s2t6u9y1-fce131d5.ondevbook.com
Open ports: [49982, 22, 53, 49982]
Open ports: [22, 49982, 8000, 49982, 53]
Open ports: [49982, 49982, 22, 8000, 53]
Open ports: [49982, 49982, 22, 8000, 53]
Open ports: [22, 53, 49982, 49982, 8000]
Open ports: [49982, 8000, 49982, 53, 22]
Open ports: [22, 49982, 8000, 53, 49982]
Open ports: [49982, 53, 49982, 8000, 22]
Open ports: [49982, 49982, 22, 8000, 53]
Open ports: [49982, 49982, 8000, 22, 53]


In [20]:
# One line initialization
sandbox = Sandbox.create(id)

proc = sandbox.process.start(
    "pwd",
    cwd="/code",
)

# You can access `proc.output...` even before the process finishes
print("Current output", proc.output.messages)

# You can wait for the result
output = proc.wait()

# List of all output messages
print("Output", output.messages)
# Concatenated stdout
print("Stdout", output.stdout)
# Concatenated stderr
print("Stderr", output.stderr)

sandbox.close()

Current output []
Output [ProcessMessage(line='/code', error=False, timestamp=1691532768216519914)]
Stdout /code
Stderr 
